# Encoder

<img src="images/transformer-architecture-5.png" width="500">

## Transformer Neural Network Architecture Overview (Encoder)

Similar to the Transformer architecture overview section in the [positional encoding](3-Positional_Encoding_in_Transformer_Neural_Network.ipynb) and [layer normalisation](4-Layer_Normalisation.ipynb) notebooks, we will be taking a quick walkthrough of the architecture once again before we discuss about the encoder in full.

Once again, we first have an input sentence, *My name is John*. We first pad the input sequence to the maximum sequence length with dummy tokens. Each word is then represented as a one-hot encoded vector. Even though our input sentence may have different lengths (i.e. different number of words), we will always have a consistent and fixed number of vectors we pass through the Transformer.

These one-hot encoded vectors though, don't encapsulate words in a very compressed space (i.e. the vectors make up a sparse matrix) and so, in order to have more condensed vectors that incorporate better meaning so we can compute distances between these vectors, we can transform them into embedding vectors. We can do so by mapping these one-hot encoded vectors to 512-dimensional word vectors to form matrix $X$ by passing them through a transformation with learnable parameters. These parameters are learnable via back propagation that comes from the end of the decoder.

Because all words are passed into the Transformer in parallel, there is no sense of ordering. However, English sentences have words that are ordered specifically. So, we pass in some [positional encoding](3-Positional_Encoding_in_Transformer_Neural_Network.ipynb) to encode orders. We then add the input to the encoding to get the positionally encoded vectors to form matrix $X^{1}$.

**Note:** These positional encodings are a set of $\sin$ and $\cos$ functions which are not learnable and are predetermined.

After the positional encoding step is the start of the encoder layer in the Transformer. Each word vectors in the matrix $X^{1}$ is then mapped to a query, key and value vector — each of these are 512-dimensional vectors — by passing the vectors through the query, key and value weights. These weights, $W_{Q}$, $W_{K}$ and $W_{V}$ are learnable parameters. So, we are going to end up with 3 $\times$ the maximum sequence length — it is basically 3 three $\times$ the number of words in the input sequence. 

We now split each of these query, key and value vectors into 8 parts and each part (highlighted in yellow) is going to be a vector for one attention head — there are 8 attention heads in the main paper. What attention heads do is that it basically acts like an additional batch dimension (i.e. in code, it acts like another batch dimension) so that there's parallel processing that goes on and these heads can eventually interact with one another to get better context of the data. So, it's faster and also helps get better context and that's why we use [multi-head attention](2-Multihead_Attention.ipynb) versus just a [single head](1-Self_Attention_for_Transformers_Neural_Network.ipynb).

For each attention head, we compute a query, key and value vector. The query vector for every word is essentially what I am looking for. The key vector on the other hand is what can I offer and the value vector is what I actually offer. We then multiply the query and key vectors for each head, apply scaling and masking (only for decoder), to form the attention blocks, $a_{i}$ for each head. The attention blocks have the size of maximum sequence length $\times$ maximum sequence length and they tell us exactly how much attention each word should pay to the other words.

At this point, we are just right past the multi-head attention block before the `Add & Norm` block (see figure above). In the `Add & Norm` block, we will add the result from the multi-head attention block along with a skip/residual connection and perform [layer normalisation](4-Layer_Normalisation.ipynb). For very deep networks, the back propagation of values will eventually or might lead to very small gradients. This is because the activation functions like $\text{ReLU}$ or $\text{ELU}$ produce values that are either zero or values near zero. If you try to take the gradients of these values, it is going to be even smaller. That gradient is going to decrease more as we go further and further back in the network which, for longer and deeper networks, can eventually become zero. If the gradients become zero, no parameters can be updated properly and the network does not learn. This is a famous problem known as vanishing gradients. To prevent vanishing gradients in deeper networks, we tend to use residual connections.

Next, we are using layer normalisation (with learnable parameters) because we want to perform stable training. The values we might get after multi-head attention may have a wide variance and a scattered mean. With layer normalisation, we can ensure these values are centered around 0 and have unit variance 1. This also ensures even steps are taken during parameters update and the learning process.

After layer normalisation, we pass the vectors through a linear layer with $\text{ReLU}$ activation and dropout. The linear layer is going to help better interactions especially among the 8 attention blocks we previously concatenated. The $\text{ReLU}$ activation function helps the network understand better and more complex patterns and dropout acts as a regulariser by "switching off" neurons randomly which helps the network better generalise (makes sure the network does not specifically memorise a specific kind of pattern). The output matrix will be of size maximum sequence length $\times$ 1024 (or some other hidden dimension).

The output matrix is then passed into another linear layer to compress the hidden dimensions back into 512-dimensions and we are going to use another residual connection to add the matrix from the first layer normalisation to our output matrix.

Finally, we take the resulting matrix and perform the second and last layer normalisation to get the final matrix of size maximum sequence length $\times$ 512.

**Note:** Because language is so complicated, the entire encoder process (highlighted in orange in the diagram below) which we just decribed above, is carried out 12 times. All of these operations are cascaded one after the other and so that just helps us get the best vectors that represent the English words. 

Now, when we have all of these vectors, we can eventually pass all of them into the decoder that will now assist in translation from English to say, French.

![diagram](images/encoder.png)

## Import `Encoder` Class and Functions 

In [1]:
import torch
from encoder import Encoder

### Set parameters and variables

In [2]:
batch_size = 30
max_sequence_length = 200
d_model = 512
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 5

### Instantiate an `Encoder` object with parameters set above

In [3]:
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

### Test

In [4]:
x = torch.randn(
    batch_size, max_sequence_length, d_model
)  # Includes positional encoding

x.size()

torch.Size([30, 200, 512])

In [5]:
out = encoder(x)

------- ATTENTION 1 ------
x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q.size(): torch.Size([30, 8, 200, 64]), k.size(): torch.Size([30, 8, 200, 64]), v.size(): torch.Size([30, 8, 200, 64])
scaled.size() : torch.Size([30, 8, 200, 200])
values.size(): torch.Size([30, 8, 200, 64]), attention.size():torch.Size([30, 8, 200, 200])
values.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- ADD AND LAYER NORMALIZATION 1 ------
mean.size(): (torch.Size([30, 200, 1]))
var.size(): (torch.Size([30, 200, 1]))
std.size(): (torch.Size([30, 200, 1]))
y.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- DROPOUT 1 ------
------- FEED FORWARD ------
x.size() after 1st linear layer: torch.Size([30, 200, 2048])
x.size() after activation: torch.Size([30, 200, 2048])
x.size() after dropout: torch.Size([30, 200, 2048])
x.size() after